## Este Notebook tem como objetivo:
* ### extrair o texto das páginas de cada semente, de 10 sites. São eles:
  1. Syngenta
  2. NK sementes (também são da Syngenta)
  3. TMG
  4. Don Mario cerrado
  5. Don Mario Sul
  6. Brasmax Cerrado
  7. Brasmax Sul
  8. Nidera
  9. Brevant
  10. KWS

In [5]:
url_1 = 'https://www.portalsyngenta.com.br/sementes/syngenta-soja'
url_2 = 'https://www.portalsyngenta.com.br/sementes/nk-soja'
url_3 = 'https://www.tmg.agr.br/ptbr/cultivares/soja'
url_4 = 'https://www.donmario.com/pt-br/catalogo-cerrado/'
url_5 = 'https://www.donmario.com/pt-br/catalogo-sul/'
url_6 = 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado/'
url_7 = 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul/'
url_8 = 'https://www.niderasementes.com.br/soja/'
url_9 = 'https://www.brevant.com.br/produtos/soja.html'
url_10 = 'https://www.kws.com/br/pt/produtos/soja/portfolio-completo-soja/'

In [6]:
from html2text import html2text
from urllib.request import urlopen
from urllib.parse import urljoin
from bs4 import BeautifulSoup

# 1. Syngenta
* Analisando o html da página, vemos que todos os cards das sementes tem:
  * < div class = "card-text-2 card-portfolio"
  
* Para obter os links específicos de cada semente, temos que usar `urljoin`
  * link recolhido no html + link principal

* Entretanto, alguns links parciais estão incompletos.
* Para corrigir o erro, temos que usar um if dentro da def `scrap_and_colect_syngenta` para adicionar a parte que falta _(sementes/)_, caso não tenha.

In [8]:
def scrap_and_colect_syngenta(url, class_card):
    
    """Abre o link principal das sementes de soja da Syngenta e da NK sementes.
    Recebe:
    * <url> a ser scrapped; 
    * <class_card> que contém os links das sementes.
    ---> Retorna uma lista com os links de cada semente"""
    
    html = urlopen(url)
    scrap = BeautifulSoup(html, 'html.parser')

    cards = scrap.find_all('div', {'class': class_card})
    
    links = []

    for link in cards:
        link = link.a['href']
        if 'sementes' in link:
            links.append(urljoin('https://www.portalsyngenta.com.br', link))
        else: 
            link = 'sementes/'+link
            links.append(urljoin('https://www.portalsyngenta.com.br', link))
    
    return links                   

In [9]:
links_1 = scrap_and_colect_syngenta(url_1, "card-text-2 card-portfolio") 
links_1    # link para cada semente de soja Syn 

['https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn15600-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn16861-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1059-rr',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1163-rr',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1258-rr',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1281-rr',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1683-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1785-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1687-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn13610-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn15640-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn15630-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1562-ipro',
 'https://www.portalsyngenta.com.br/semen

# 2. NK sementes (Syngenta)
* Analisando o html da página, vemos que todos os cards das sementes tem:
  * < div class = "card-text-2 card-portfolio"

In [10]:
links_2 = scrap_and_colect_syngenta(url_2, "card-text-2 card-portfolio") 
links_2    # link para cada semente de soja NK

['https://www.portalsyngenta.com.br/sementes/nk-soja/nk-8770-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-7777-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-8448-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-8301-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-7201-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-6201-ipro']

# 3. TMG
* Analisando o html da página, vemos que s cards das sementes tem essa div, porém com CSS **FLEX container**:
  * < div class = "cultivar-link-v" flex
* No caso da TMG, os links não são parciais, já vem completos.
* Por isso a função pode ser diferente, sem o urljoin.
* Entretanto, com a página em .js dinâmica, temos que usar `selenium` e `chromedriver`
  * REF: https://stackoverflow.com/questions/62165635/how-to-scrape-data-from-flexbox-element-container-with-python-and-beautiful-soup
  * download Chromedrive: https://chromedriver.storage.googleapis.com/index.html?path=90.0.4430.24/
    * O caminho do driver dentro do pc será usado. 
    * Exemplo: /home/anandaheino/chromedriver
    
## * função que extrai conteúdo dinâmico: 


In [11]:
# !pip install selenium

from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from time import sleep

In [12]:
def scrap_and_colect_css(url, class_card):
    
    """Abre o link principal das sementes de soja.
    Usa <div>.
    Recebe:
    * <url> a ser scrapped; 
    * <class_card> que contém os links das sementes.
    ---> Retorna uma lista com os links de cada semente"""

    
    chrome_options = Options()
    
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument('--disable-infobars')
    chrome_options.add_argument("disable-infobars")
    chrome_options.add_experimental_option("prefs", 
                                           {"profile.default_content_setting_values.notifications": 2})
    
    browser = webdriver.Chrome(executable_path='/home/anandaheino/chromedriver', options = chrome_options) 
    browser.get(url)
   
    delay = 5 #secods

    #try:
    WebDriverWait(browser, delay)
    print ("\nPage is ready")
    sleep(5)
    browser.minimize_window()
    html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")

    #print(html)
    scrap = BeautifulSoup(html, "html.parser")
    cards = scrap.find_all('div', {'class': class_card})

    links = list(link.a['href'] for link in cards)
    return links

    browser.close() 
    browser.quit()
    
    #except TimeoutException:
        #print ("Loading took too much time! -Try again")

    #finally:
        #browser.quit()
        

In [13]:
links_3 = scrap_and_colect_css(url_3, "cultivar-link-v") 
links_3    # link para cada semente de soja da TMG


Page is ready


['https://www.tmg.agr.br/ptbr/cultivar/tmg-2375-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2379-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-7058-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2165-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-4377',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-7368-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2757-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2776-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2374-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2359-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2285-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-1180-rr',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-7063-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-7062-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-7067-ipro',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-4182',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-7262-rr',
 'https://www.tmg.agr.br/ptbr/cultivar/tmg-2185-ipro',
 'https://www.tmg.agr.br

# 4. Don Mario - Cerrado
* Analisando o html da página, vemos que os cards das sementes são:
  * < div class="images_holder"
* No caso da Don Mario, os links não são parciais, já vem completos.
* Por isso a função pode ser mais simples.

### * Implementando uma Função simples, que retornas os links dos cards informados, temos o seguinte erro:
      * HTTPError: HTTP Error 403: Forbidden

### * Usando a função com Selenium, foi possível extrair os links dos cards das sementes.

In [14]:
links_4 = scrap_and_colect_css(url_4, "images_holder")
links_4


Page is ready


['https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM80IX83',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM75i74',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM81I84',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM82K84',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM82i78',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM75i76',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM79I81',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM80i79',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM73i75',
 'https://www.donmario.com/pt-br/cultivares-cerrado/#section-DM68i69']

# 5. Don Mario - Sul
* Analisando o html da página, vemos que os cards das sementes são iguais do Cerrado:
  * < div class="images_holder"

In [15]:
links_5 = scrap_and_colect_css(url_5, "images_holder")
links_5


Page is ready


['https://www.donmario.com/pt-br/cultivares-sul/#section-DM64I63IPRO',
 'https://www.donmario.com/pt-br/cultivares-sul/#section-DM53i54',
 'https://www.donmario.com/pt-br/cultivares-sul/#section-DM60I63IPRO',
 'https://www.donmario.com/pt-br/cultivares-sul/#section-DM5958',
 'https://www.donmario.com/pt-br/cultivares-sul/#section-DM66i68',
 'https://www.donmario.com/pt-br/cultivares-sul/#section-DM57i52']

# 6. Brasmax - Cerrado
* Analisando o html da página, vemos que os cards das sementes tem:

* <a class="info" href="https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=xxxxx"
     
### * Implementando uma Função simples,temos o seguinte erro:
      * HTTPError: HTTP Error 403: Forbidden

### * Usando a função com Selenium, foi possível extrair os links dos cards das sementes.

In [17]:
def scrap_and_colect_css_Brasmax(url, class_card):
    
    """Abre o link principal das sementes de soja.
    Usa <a>.
    Recebe:
    * <url> a ser scrapped; 
    * <class_card> que contém os links das sementes.
    ---> Retorna uma lista com os links de cada semente"""

    
    chrome_options = Options()
    
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument('--disable-infobars')
    chrome_options.add_argument("disable-infobars")
    chrome_options.add_experimental_option("prefs", 
                                           {"profile.default_content_setting_values.notifications": 2})
    
    browser = webdriver.Chrome(executable_path='/home/anandaheino/chromedriver', options = chrome_options) 
    browser.get(url)
   
    delay = 5 #secods

    #try:
    WebDriverWait(browser, delay)
    print ("\nPage is ready")
    sleep(5)
    browser.minimize_window()
    html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")

    #print(html)
    scrap = BeautifulSoup(html, "html.parser")
    cards = scrap.find_all('a', {'class': class_card})
    # print(cards)
    links = list(link.get('href') for link in cards)
    
    return links

    browser.close() 
    
    #except TimeoutException:
        #print ("Loading took too much time! -Try again")

    #finally:
        #browser.quit()
        

In [18]:
links_6 = scrap_and_colect_css_Brasmax(url_6, "info")
links_6


Page is ready


['https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=14243',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=14248',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=14257',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=14234',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=14253',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=14262',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=14239',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=244',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=258',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=273',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=4985',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=8916',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=969',
 'https:/

# 7. Brasmax - Sul
* Analisando o html da página, vemos que os cards das sementes são iguais ao do cerrado:

  * <a class="info" href="https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado?produto=xxxxx"
     


In [19]:
links_7 = scrap_and_colect_css_Brasmax(url_7, "info")
links_7


Page is ready


['https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=14276',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=14280',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=14267',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=14284',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=14271',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=2501',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=4920',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=1113',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=4927',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=1133',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=1103',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=1149',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?produto=2496',
 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul?pr

# 8. Nidera
* Analisando o html da página, vemos que os cards das sementes são:
  * div class="title"

* Implementando a função `scrap_and_colect_css` para coletar os links das sementes


In [20]:
links_8 = scrap_and_colect_css(url_8, 'title')
links_8


Page is ready


['https://www.niderasementes.com.br/soja_detalhe/ns-8109-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-7790-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-7654-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-8595-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-8400-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-8590-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-8397-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-8338-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-8300-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-7700-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-8399-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-7709-ipro-centro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-7780-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-7667-ipro/',
 'https://www.niderasementes.com.br/soja_detalhe/ns-7505-ipro/',
 'https://www.nide

In [21]:
len(links_8)

47

# 9. Brevant
* Analisando o html da página, vemos que os cards das sementes são:
  * td class = "body-cell-value"
  * os links disponibilizados **não** estão completos. 
  * usamos `urljoin` para juntar com o link principal da página.

### * refatorando a função `scrap_and_colect_css` para este caso, temos:

In [22]:
url_brevant = 'https://www.brevant.com.br'
driver = webdriver.Chrome(executable_path='/home/anandaheino/chromedriver')
driver.get(url_9)
cookies_9 = driver.get_cookies()
for cookie in cookies_9:
    driver.add_cookie(cookie)
# driver.find_element_by_xpath('/html/body/div[8]/div[1]/div/div[3]/a[1]')

In [120]:
len(cookies_9)

7

In [34]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

def scrap_and_colect_css_Brevant(url, class_card):
    
    """Abre o link principal das sementes de soja.
    Usa <td> 
    Recebe:
    * <url> a ser scrapped; 
    * <class_card> que contém os links das sementes.
    ---> Retorna uma lista com os links de cada semente"""

    
    chrome_options = Options()
    
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument('--disable-infobars')
    chrome_options.add_argument("disable-infobars")
    chrome_options.add_experimental_option("prefs", 
                                           {"profile.default_content_setting_values.notifications": 2})
    
    browser = webdriver.Chrome(executable_path='/home/anandaheino/chromedriver', options = chrome_options) 
    browser.get(url)
    
    # tentativa de adicionar os cookies do site na lista de cookies do selenium
    cookies = browser.get_cookies()
    for cookie in cookies:
        browser.add_cookie(cookie)
    
    try:
        WebDriverWait(browser, 5)
        print ("\nPage is ready")
        sleep(5)

        html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")

        # print(html)
        scrap = BeautifulSoup(html, "html.parser")
        cards = scrap.find_all("a", href=re.compile("produtos/soja/"))
        # print(cards[0:5])
        
        links = []

        for link in cards:
            link = link['href']
            # print(link)
            links.append(urljoin(url_brevant, link))
            
        return links

        browser.close() 
        browser.quit()
        
    except TimeoutException:
        print ("Loading took too much time! -Try again")


In [35]:
links_9 = scrap_and_colect_css_Brevant(url_9,"body-cell-value")
links_9


Page is ready


['https://www.brevant.com.br/produtos/soja/bg4290.html',
 'https://www.brevant.com.br/produtos/soja/bg4786.html',
 'https://www.brevant.com.br/produtos/soja/cd2720ipro.html',
 'https://www.brevant.com.br/produtos/soja/cd2728ipro.html',
 'https://www.brevant.com.br/produtos/soja/cd2820ipro.html',
 'https://www.brevant.com.br/produtos/soja/cd2851ipro.html',
 'https://www.brevant.com.br/produtos/soja/cd2857rr.html',
 'https://www.brevant.com.br/produtos/soja/ds5916ipro.html',
 'https://www.brevant.com.br/produtos/soja/5d6215ipro.html',
 'https://www.brevant.com.br/produtos/soja/5d634rr.html',
 'https://www.brevant.com.br/produtos/soja/5g8015ipro.html',
 'https://www.brevant.com.br/produtos/soja/ds6017ipro.html',
 'https://www.brevant.com.br/produtos/soja/ds6217ipro.html',
 'https://www.brevant.com.br/produtos/soja/ds7417ipro.html',
 'https://www.brevant.com.br/produtos/soja/b5860ipro.html',
 'https://www.brevant.com.br/produtos/soja/bg4781ipro.html',
 'https://www.brevant.com.br/produtos/